In [1]:
import torch.nn as nn
import torch

In [4]:
index = 1
while True:
    a = nn.Linear(10000,100*index)
    a.cuda()
    index +=1 

KeyboardInterrupt: 

In [4]:
linear = nn.Linear(2,4)

In [10]:
linear.weight

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], requires_grad=True)

In [9]:
nn.init.zeros_(linear.weight)

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], requires_grad=True)

In [16]:
class MOELoraLayer(nn.Module):
    def __init__(self, dim, r, expert_num, hydra=False):
        super().__init__()
        self.expert_num = expert_num
        self.hydra = hydra # hydra lora

        self.router = nn.Linear(dim, expert_num, bias=False)

        if hydra:
            self.lora_A = nn.Linear(dim, r, bias=False)
        else:
            self.lora_A = nn.ModuleList()
            for i in range(expert_num):
                self.lora_A.append(nn.Linear(dim, r, bias=False))
            
        self.lora_B = nn.ModuleList()
        for i in range(expert_num):
            self.lora_B.append(nn.Linear(r, dim, bias=False))

        # initial lora B to zeros
        for linear in self.lora_B:
            nn.init.zeros_(linear.weight)

    def forward(self, x: torch.Tensor):
        route_weight = nn.functional.softmax(self.router(x), dim=-1, dtype=torch.float32).to(x.dtype)
        # try lora_alpha
        for i in range(self.expert_num):
            if self.hydra:
                x = x + torch.unsqueeze(route_weight[:,:,i], -1) * self.lora_B[i](self.lora_A(x))
            else:
                x = x + torch.unsqueeze(route_weight[:,:,i], -1) * self.lora_B[i](self.lora_A[i](x))
        return x

In [17]:
moelora = MOELoraLayer(10,2,5)

In [29]:
moelora.state_dict().keys()

odict_keys(['router.weight', 'lora_A.0.weight', 'lora_A.1.weight', 'lora_A.2.weight', 'lora_A.3.weight', 'lora_A.4.weight', 'lora_B.0.weight', 'lora_B.1.weight', 'lora_B.2.weight', 'lora_B.3.weight', 'lora_B.4.weight'])

In [24]:
x = torch.randn(3,4)
a = torch.randn(4,2)
a1= torch.randn(4,2)
a2= torch.randn(4,2)

b1 = torch.randn(2,4)
b2= torch.randn(2,4)

relu = nn.ReLU()

In [20]:
(relu(x @ a1) @ b1)*0.4 + (relu(x @ a2) @ b2)*0.6

tensor([[ 0.0131,  0.8114, -0.1674,  0.1921],
        [ 0.3173,  1.5902,  1.2141,  0.6184],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [21]:
relu(x @ a1) @ (b1*0.4) + relu(x @ a2) @ (b2*0.6)

tensor([[ 0.0131,  0.8114, -0.1674,  0.1921],
        [ 0.3173,  1.5902,  1.2141,  0.6184],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [25]:
relu(x @ a) @ ( b1*0.4) + relu(x @ a) @ (b2*0.6)

tensor([[-0.5762, -0.2077, -0.6032, -0.1846],
        [-0.3817, -0.1376, -0.3996, -0.1223],
        [ 0.0645, -0.0288,  0.0293, -0.0455]])

In [26]:
relu(x @ a) @ ( b1*0.4 + b2*0.6)

tensor([[-0.5762, -0.2077, -0.6032, -0.1846],
        [-0.3817, -0.1376, -0.3996, -0.1223],
        [ 0.0645, -0.0288,  0.0293, -0.0455]])

In [2]:
a = [1,2,3,4]
ta = torch.FloatTensor(a)

In [8]:
ta.dtype

torch.float32

In [6]:
nn.functional.sigmoid(ta)

tensor([0.7311, 0.8808, 0.9526, 0.9820])

In [12]:
nn.functional.softmax(ta)*4


/tmp/ipykernel_1876773/2441912604.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.functional.softmax(ta)*4


tensor([0.1282, 0.3486, 0.9475, 2.5757])

In [14]:
ta.unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 4])

In [2]:
a = torch.randn(3,4,2)

In [3]:
b = torch.randn(3,4,2,5)

In [9]:
torch.sum(a.unsqueeze(-1) * b, 2, keepdim=True).shape

torch.Size([3, 4, 1, 5])

In [4]:
a == None

False

# math length

In [4]:
from LLaMA3_lora_bias.llama import Tokenizer
model_path = '/home2/caojie/pretrain_models/Meta-Llama-3-8B/'
tokenizer = Tokenizer(model_path= f"{model_path}/tokenizer.model")

/home2/caojie/miniconda3/envs/summary/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


------ flash attention2 enable -----


In [13]:
import json
# with open(f'/home2/caojie/projects/LLM-Adapters/ft-training_set/math_14k.json', 'r') as f:
# with open(f'/home2/caojie/projects/LLM-Adapters/ft-training_set/commonsense_15k.json', 'r') as f:
with open(f'/home2/caojie/datasets/math_commonsense/hellaswag/test.json', 'r') as f:
    data = f.read()

data = json.loads(data)

In [14]:
data[1]

{'instruction': 'Please choose the correct ending to complete the given sentence: Clean and jerk: A lady walks to a barbell. She bends down and grabs the pole. the lady\n\nEnding1: swings and lands in her arms. Ending2: pulls the barbell forward. Ending3: pulls a rope attached to the barbell. Ending4: stands and lifts the weight over her head.\n\nAnswer format: ending1/ending2/ending3/ending4',
 'input': '',
 'output': 'the correct answer is ending4',
 'answer': 'ending4'}

In [15]:
input_lens=[]
output_lens=[]
for x in data:
    input_lens.append(len(tokenizer.encode(x['instruction']+x['output'], bos=False, eos=False)))
    output_lens.append(len(tokenizer.encode(x['output'], bos=False, eos=False)))
print(f'average tokens:{sum(input_lens)/len(input_lens)}')
print(f'average tokens:{sum(output_lens)/len(output_lens)}')

average tokens:223.45678151762598
average tokens:6.0


In [16]:
max(input_lens)


391

In [17]:
len(input_lens)

10042

In [1]:
a = {'a':1, 'b':2}

In [7]:
sum(a.values())/len(a.values())

1.5

In [1]:
list(range(0,32))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [2]:
a = torch.randn(2,4)

In [4]:
a.unsqueeze(1).shape

torch.Size([2, 1, 4])

In [2]:
a = torch.nn.Parameter(torch.zeros(1))

In [3]:
b = torch.randn(3,4)

In [4]:
a * b

tensor([[-0., -0., 0., -0.],
        [0., 0., -0., -0.],
        [0., 0., -0., -0.]], grad_fn=<MulBackward0>)

In [1]:
a = [1,2,3]

In [2]:
a[1:None]

[2, 3]

In [2]:
import torch

In [9]:
a = torch.nn.Parameter(torch.zeros(3,4).unsqueeze(0))

In [6]:
a.device

device(type='cpu')

In [11]:
torch.nn.init.normal_(a.data)

tensor([[[-1.4849, -0.9344,  1.2820, -0.6792],
         [ 1.1104, -2.3331,  1.0779,  1.6265],
         [-1.2120,  0.3349, -0.3242,  1.1450]]])

In [12]:
a

Parameter containing:
tensor([[[-1.4849, -0.9344,  1.2820, -0.6792],
         [ 1.1104, -2.3331,  1.0779,  1.6265],
         [-1.2120,  0.3349, -0.3242,  1.1450]]], requires_grad=True)

In [1]:
import torch
import torch.nn as nn

In [3]:
bsz = 3
seqlen= 4
num_experts = 5

In [4]:
logits = torch.randn(bsz, seqlen, num_experts)

In [5]:
selected_experts = torch.ge(logits, 0).to(torch.float)

In [6]:
selected_experts

tensor([[[0., 0., 1., 0., 0.],
         [0., 0., 1., 1., 0.],
         [0., 1., 0., 1., 0.],
         [0., 1., 0., 0., 1.]],

        [[1., 0., 0., 0., 0.],
         [0., 1., 1., 1., 1.],
         [1., 1., 0., 0., 1.],
         [0., 0., 1., 1., 0.]],

        [[1., 0., 1., 1., 0.],
         [1., 1., 0., 0., 1.],
         [1., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.]]])

In [8]:
logits

tensor([[[-0.9918, -1.0010,  0.2438, -0.4004, -1.0390],
         [-0.3804, -0.7585,  0.1342,  1.3816, -1.9064],
         [-1.7549,  1.2940, -1.7741,  1.3281, -0.6339],
         [-2.1895,  0.6686, -0.2083, -0.9226,  0.2544]],

        [[ 1.2363, -0.7691, -1.6944, -1.7630, -0.5213],
         [-0.1724,  1.1549,  0.1121,  0.6297,  1.0693],
         [ 0.0719,  0.4409, -0.7988, -0.5859,  0.3340],
         [-2.1560, -0.1802,  0.5282,  1.5147, -1.0501]],

        [[ 1.2970, -1.4030,  0.5333,  0.5362, -0.1843],
         [ 0.3354,  0.7493, -0.6685, -0.1041,  2.3919],
         [ 0.6076, -0.4399, -1.7465, -0.1077,  0.7130],
         [-0.9992, -0.2151, -0.5162, -0.4907, -1.8281]]])

In [9]:
logits = logits * selected_experts

In [15]:
expert0 = logits[:,:,0]

In [39]:
expert0

tensor([[-0.0000, -0.0000, -0.0000, -0.0000],
        [1.2363, -0.0000, 0.0719, -0.0000],
        [1.2970, 0.3354, 0.6076, -0.0000]])

In [45]:
torch.where(expert0)

(tensor([1, 1, 2, 2, 2]), tensor([0, 2, 0, 1, 2]))

In [24]:
input = torch.randn(bsz, seqlen, 2)

In [44]:
input[torch.where(expert0)]

tensor([[ 1.1565, -1.1045],
        [ 0.0485,  0.1690],
        [ 0.1620, -1.7139],
        [ 0.3355,  1.9694],
        [ 0.9939, -1.0204]])

In [32]:
len(c[0])

0

In [41]:
expert0[torch.where(expert0)]

tensor([1.2363, 0.0719, 1.2970, 0.3354, 0.6076])

In [34]:
logits.shape

torch.Size([3, 4, 5])

In [43]:
expert0[torch.where(expert0)].unsqueeze(-1) * input[torch.where(expert0)]

tensor([[ 1.4297, -1.3655],
        [ 0.0035,  0.0122],
        [ 0.2101, -2.2229],
        [ 0.1125,  0.6605],
        [ 0.6039, -0.6201]])